In [ ]:
import os

In [ ]:
# Current folder path
%pwd

In [ ]:
# Go to the roor directory and give the path
os.chdir("../")
%pwd

In [ ]:
from dataclasses import dataclass
from pathlib import Path
from constants import *
from utils.base_utils import read_yaml, create_directories

In [ ]:
# == Entity ==
@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_augmentation: bool
    params_image_size: list
    params_learning_rate: float

In [ ]:
# Configuration Manager ==
class ConfigurationManager:

    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_path=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_path)

        create_directories([self.config.artifacts_root])

    def get_training_config(self) -> TrainingConfig:
        training_config = self.config.training
        prepare_base_model_config = self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(
            self.config.data_preprocessing.root_dir, "Food_Spoilage_Dataset"
        )

        create_directories([training_config.root_dir])

        training_config = TrainingConfig(
            root_dir=Path(training_config.root_dir),
            trained_model_path=Path(training_config.trained_model_path),
            updated_base_model_path=Path(
                prepare_base_model_config.updated_base_model_path
            ),
            training_data=Path(training_data),
            params_epochs=params.EPOCHS,
            params_batch_size=params.BATCH_SIZE,
            params_is_augmentation=params.AUGMENTATION,
            params_image_size=params.IMAGE_SIZE,
            params_learning_rate=params.LEARNING_RATE,
        )

        return training_config

In [ ]:
# == Componets ==

import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [ ]:
class Training:
    def __init__(self, config: TrainingConfig):
        self.config = config

    def get_based_model(self):
        self.model = tf.keras.models.load_model(self.config.updated_base_model_path)

    def train_valid_generator(self):
        # Normalize images and keep 20% aside for validation
        datagen_kwargs = dict(rescale=1.0 / 255, validation_split=0.20)

        # Image loading settings
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],  # Image size model expects
            batch_size=self.config.params_batch_size,  # Images per training step
            class_mode="categorical",  # For multi-class classification
            interpolation="bilinear",  # Interpolation means how the images are resized
        )

        # Augmentation settings for training generator
        if self.config.params_is_augmentation:
            train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagen_kwargs
            )
        else:
            # Create Training Generator
            train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                **datagen_kwargs
            )

        self.train_generator = train_datagen.flow_from_directory(
            directory=self.config.training_data,
            subset="training",
            shuffle=True,
            **dataflow_kwargs
        )

        # Create Validation Generator
        valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagen_kwargs
        )

        self.valid_generator = valid_datagen.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    def train(self):
        # Compile the based model
        self.model.compile(
            optimizer=tf.keras.optimizers.legacy.Adam(
                learning_rate=self.config.params_learning_rate
            ),
            loss="categorical_crossentropy",
            metrics=[
                "accuracy",
                tf.keras.metrics.Precision(name="precision"),
                tf.keras.metrics.Recall(name="recall"),
                tf.keras.metrics.AUC(name="auc"),
            ],
        )

        callbacks = [
            tf.keras.callbacks.EarlyStopping(
                monitor="val_loss", patience=5, restore_best_weights=True
            )
        ]

        # Train the model
        self.model.fit(
            self.train_generator,
            epochs=self.config.params_epochs,
            validation_data=self.valid_generator,
            callbacks=callbacks,
        )

    def save_model(self):
        if self.model is None:
            raise ValueError("Model not trained or loaded yet")

        path = self.config.trained_model_path.with_suffix(".keras")
        self.model.save(path)

In [ ]:
# Pipeline
try:
    # Initilize the ConfigurationManager
    config = ConfigurationManager()
    # Get the config yaml file details
    training_config = config.get_training_config()
    # Initialize the Trainer
    trainer = Training(config=training_config)
    # Get the based model
    trainer.get_based_model()
    # Get the train and valid generator
    trainer.train_valid_generator()
    # Train the model
    trainer.train()
    # Save the trained model
    trainer.save_model()
except Exception as e:
    raise e